In [1]:
import os
import numpy as np
import random
import glob
import warnings
import shutil
import tqdm
from pathlib import Path
from terminaltables import AsciiTable

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.transforms import functional as F

from pytorchyolo.models import load_model
from pytorchyolo.train import _create_data_loader
from pytorchyolo.test import _create_validation_data_loader, _evaluate

from pytorchyolo.utils.transforms import *
from pytorchyolo.utils.augmentations import AUGMENTATION_TRANSFORMS
from pytorchyolo.utils.optimizer import get_optimizer
from pytorchyolo.utils.datasets import *
from pytorchyolo.utils.logger import Logger
from pytorchyolo.utils.loss import compute_loss
from pytorchyolo.utils.parse_config import parse_data_config
from pytorchyolo.utils.utils import load_classes,provide_determinism,to_cpu

In [7]:
class CFG:
    if torch.cuda.is_available():
      device = 'cuda'  
      NUM_DEVICES = torch.cuda.device_count()
    elif torch.backends.mps.is_available():
      device = 'mps'
      NUM_DEVICES = torch.mps.device_count()
    else:
      device ='cpu' 
      NUM_DEVICES = os.cpu_count()
      
    n_cpu=  os.cpu_count()
    mini_batch_size = 8
    multiscale_training = False
    logdir = "/Users/ayyappatata/DL_P/2/PyTorch-YOLOv3/logs/"
    epochs = 5
    seed = 1
    checkpoint_interval = 5
    evaluation_interval = 5
    

In [8]:
data_config = parse_data_config("./config/custom.data")
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

In [3]:
transforms_testing = transforms.Compose([
    ToTensor()
])

In [6]:
model = torch.load("fzd_model.pt")

/var/folders/4y/b_h0g7n103345tnskjhk1_s00000gn/T/ipykernel_49696/3541142265.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("fzd_model.pt")


In [10]:
dataloader = _create_data_loader(
    img_path=train_path,
    img_size=416,
    batch_size=CFG.mini_batch_size,
    n_cpu = CFG.n_cpu)

In [11]:
DEFAULT_TRANSFORMS

Compose(
)

In [12]:
dataset = ListDataset(
        train_path,
        img_size=416,
        multiscale=True,
        transform=transforms_testing
        )